In [1]:
from aiida.backends.utils import load_dbenv, is_dbenv_loaded

if not is_dbenv_loaded():
    load_dbenv()

from aiida.work.run import run, submit
from aiida.work.workfunction import workfunction
from aiida.work.workchain import WorkChain, ToContext, while_, Outputs, if_, append_
from aiida.orm.data.base import Float, Str, NumericType, BaseType, Int, Bool, List
from aiida.orm.code import Code
from aiida.orm import load_node

from aiida.work.workchain import WorkChain, ToContext, if_, while_, Outputs, if_, append_

from aiida.orm.data.base import Float, Str, NumericType, BaseType, Int, Bool, List

from aiida.orm.data.parameter import ParameterData
from aiida.orm.data.structure import StructureData
from aiida.orm.data.array.kpoints import KpointsData
from aiida.orm.data.array import ArrayData
from aiida.orm.data.folder import FolderData
from aiida.orm.data.remote import RemoteData
from aiida.orm import DataFactory
from aiida.orm.data.singlefile import SinglefileData

from aiida_defects.epsilon.bulkepsilonberry import BulkEpsilonBerryWorkChain


In [2]:
from aiida.orm import load_node
from aiida.work.run import run
from aiida.orm.data.parameter import ParameterData
from aiida.orm.data.structure import StructureData
from aiida.orm.data.array.kpoints import KpointsData
from aiida.orm.data.base import  Str, Bool, Float, Int
from aiida_defects.pp.pp import PpWorkChain

from aiida.orm import load_node
inputfile='si.cif'


from pymatgen.io import cif,aiida
from pymatgen.io.aiida import AiidaStructureAdaptor

structure_mg= cif.Structure.from_file(str(inputfile))
aiida_structure_adaptor = AiidaStructureAdaptor()
structure_sd = aiida_structure_adaptor.get_structuredata(structure_mg)
codename="pw_5.1@ubelix"
code_pp="pp_5.1@ubelix"
pseudo_family='SSSP'
#defect_position = ArrayData()
#defect_position.set_array("position", np.array([0., 0., 0.]))
defect_charge= 2.
epsilon_r =  2.



options={
        'resources': {
            'num_machines': 1,
            #'num_mpiprocs_per_machine': 1,
        },
         'max_wallclock_seconds' : 36000,
         #'custom_scheduler_commands' : u"#SBATCH --partition=all",
         #'custom_scheduler_commands' : u"#SBATCH --account=dcb",
         'custom_scheduler_commands' : u"#SBATCH --partition=empi",

        }

settings={}

kpoints = KpointsData()
kpoints.set_kpoints_mesh([3, 3, 7])
kpoints = KpointsData()
kpoints.set_kpoints_mesh([3, 3, 7])


parameters = {
        'CONTROL': {
            'restart_mode': 'from_scratch',
            #'tstress': True,
            #'tprnfor' : True,
            'nstep' : 500,
            'etot_conv_thr' : 1.0e-6,
             'forc_conv_thr' : 1.0e-3,
            #'tefield'      : True,
            #'dipfield'  : True
        },
        'SYSTEM': {
            'ecutwfc': 40.,
            'ecutrho': 320.,
            'occupations' : 'fixed',
            #'degauss' : 0.01,
            #'edir'   : 3,
            #'emaxpos' : 0.75,
            #'eopreg': 0.01,
            #'eamp' : 0.0,

            #'starting_magnetization' : starting_magnetization,
        },
        'ELECTRONS': {
            'conv_thr': 1.e-8,
            'mixing_beta' : 0.6,
            'startingwfc' : 'atomic',
        },

    }

parameters_pp = {
        'INPUTPP': {'plot_num' : 11,
        }
        }


outputs=run(BulkEpsilonBerryWorkChain,
            structure=structure_sd,
            code_pw=Str(codename),
            pseudo_family=Str(pseudo_family),
            kpoints=kpoints,
            parameters=ParameterData(dict=parameters),
            settings=ParameterData(dict=settings),
            options=ParameterData(dict=options),
           )

12/07/2018 11:50:44 AM, aiida.aiida.orm.implementation.general.calculation.work.WorkCalculation: [REPORT] [161019|BulkEpsilonBerryWorkChain|run_pw_e0]: Launching PwBaseWorkChain for a FEF calculation with amplitude 0.0 a.u.. pk value 161024
12/07/2018 12:58:12 PM, aiida.aiida.orm.implementation.general.calculation.work.WorkCalculation: [REPORT] [161019|BulkEpsilonBerryWorkChain|run_pw_e1]: Launching PwBaseWorkChain for a FEF calculation with amplitude 0.001 a.u.. pk value 161199
12/07/2018 01:01:08 PM, aiida.aiida.orm.implementation.general.calculation.work.WorkCalculation: [REPORT] [161019|BulkEpsilonBerryWorkChain|compute_bulk_epsilon]: BulkEpsilonBerryWorkChain completed succesfully. The high-frequency dielectric constant is 12.5316235962


In [65]:
#spec.input('epsilon_type', valid_type=Str, required=False, default=Str('high-frequency'))

outputs=run(BulkEpsilonBerryWorkChain,
            structure=structure_sd,
            code_pw=Str(codename),
            pseudo_family=Str(pseudo_family),
            kpoints=kpoints,
            parameters=ParameterData(dict=parameters),
            settings=ParameterData(dict=settings),
            options=ParameterData(dict=options),
            epsilon_type=Str('low-frequency')
           )

12/06/2018 04:57:41 PM, aiida.aiida.orm.implementation.general.calculation.work.WorkCalculation: [REPORT] [160074|BulkEpsilonBerryWorkChain|abort_nowait]: Aborting: The calculation of the low-frequency dielectric constant within the Berry phase method is not yet implemented
